In [1]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)


SDK version: 1.51.0


In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')


mlv2
ml_group
eastasia
956a5d16-ed62-4076-890a-4f7ea58eef95


In [4]:
from azureml.core.model import Model
# Tip: When model_path is set to a directory, you can use the child_paths parameter to include
#      only some of the files from the directory
model = Model.register(model_path = "XGBoost_ramman_model.h5",
                       model_name = "raman_model_nb",
                       workspace = ws)

model = Model.register(model_path = "scale_raman.h5",
                       model_name = "scale_raman",
                       workspace = ws)


Registering model raman_model_nb
Registering model scale_raman


In [3]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="myenv_raman")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("joblib==1.2.0")
# conda_dep.add_conda_package("keras==2.11.0")
conda_dep.add_conda_package("numpy==1.24.2")
conda_dep.add_conda_package("pandas==1.5.3")
# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("scikit-learn==1.2.1")
# conda_dep.add_pip_package("pickle")

conda_dep.add_pip_package("scipy==1.10.1")
# conda_dep.add_pip_package("tensorflow==2.11.0")
conda_dep.add_pip_package("xgboost==1.7.4")


# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="score_raman.py",
                                   environment=myenv)



In [40]:
print(service.get_logs())


/bin/bash: /azureml-envs/azureml_c894f7eb32975bcc312adae58f03236d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c894f7eb32975bcc312adae58f03236d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c894f7eb32975bcc312adae58f03236d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-09-08T12:48:31,894112698+00:00 - rsyslog/run 
bash: /azureml-envs/azureml_c894f7eb32975bcc312adae58f03236d/lib/libtinfo.so.6: no version information available (required by bash)
2023-09-08T12:48:31,922578689+00:00 - nginx/run 
2023-09-08T12:48:31,930359413+00:00 - gunicorn/run 
2023-09-08T12:48:31,941339248+00:00 | gunicorn/run | 
2023-09-08T12:48:31,952217983+00:00 | gunicorn/run | ###############################################
2023-09-08T12:48:31,978219766+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-09-08T12:48:31,987963197+00:00 | gunico

In [4]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.compute.aks import AksCompute
from azureml.core.compute import ComputeTarget

aks_target = AksCompute(ws,"dev-aks2-v1")
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

network = Model(ws, name='raman_model_nb')
scale = Model(ws, name='scale_raman')

service = Model.deploy(ws, 'raman-service-v2', [network,scale], inference_config, deployment_config, aks_target)

service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())


/tmp/ipykernel_29953/1480452124.py:12: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, 'raman-service-v2', [network,scale], inference_config, deployment_config, aks_target)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-09-11 03:35:12+00:00 Creating Container Registry if not exists.
2023-09-11 03:35:12+00:00 Registering the environment.
2023-09-11 03:35:14+00:00 Use the existing image.
2023-09-11 03:35:16+00:00 Creating resources in AKS.
2023-09-11 03:35:16+00:00 Submitting deployment to compute.
2023-09-11 03:35:16+00:00 Checking the status of deployment raman-service-v2.